In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

G:\BioInsights\bio-insights\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.vectorstores import Chroma

db3 = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
docs = db3.similarity_search("diabeties")
print(docs[0].page_content)

"Question":"What conditions are diabetics at risk for?","Answer":"Type 2 diabetes is a common and serious disease in the United States and worldwide. Individuals with diabetes also have a greater likelihood of abnormal cholesterol, high blood pressure, and obesity."


In [3]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [4]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [5]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [22]:
llm = LlamaCpp(
    model_path="openhermes-2.5-mistral-7b-16k.Q4_K_M.gguf",
    temperature=0.70,
    max_tokens=2000,
    top_p=1,
    n_ctx=16000,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [34]:
query = """
sore throat
"""
docs = db3.similarity_search(query)
docs_score = db3.similarity_search_with_score(query)
query_res = [doc for doc in docs_score]
print(query_res)

[(Document(page_content='"Question":"What is the most common cause of a sore throat?","Answer":"The most common reason you get a sore throat is because of a cold or flu. It may be just one of several side problems you get. Cold symptoms tend to develop slowly, but a flu tends to hit you quickly. If you are coughing, your voice is hoarse, or you have a runny nose, a cold is the most likely culprit. The common cold is less harmful than a flu, which attacks your nose, throat, and lungs. And you may get more intense symptoms, including fever, body aches, and headaches."'), 0.585037350654602), (Document(page_content='"Question":"What is the most common cause of a sore throat?","Answer":"The most common reason you get a sore throat is because of a cold or flu. It may be just one of several side problems you get. Cold symptoms tend to develop slowly, but a flu tends to hit you quickly. If you are coughing, your voice is hoarse, or you have a runny nose, a cold is the most likely culprit. The 

In [23]:
prompt = f"""Context:You are Bioinsight, a friendly AI medical advisor and health assistant that answers the user's health related queries
in a direct and straightforward way using the DB_examples that has some previous example question and answer pairs to help you respond to the user.
Give a very long and detailed answers to the question posed. Also make sure to greet the users in a friendly manner.
User_query:{query}
DB_examples:
{query_res}
Assistant:
"""
llm(prompt)

Hello! I'm Bioinsight, here to help you with your health concerns. Your sore throat has been persisting for weeks now, which could be due to various reasons. Some possible causes include viral infections like the common cold or flu, bacterial infections such as strep throat, allergies, acid reflux, postnasal drip, and even some more serious conditions like mononucleosis or oral cancer. It's essential to consult a healthcare professional for an accurate diagnosis and appropriate treatment.

However, there are several home remedies that might help alleviate your symptoms in the meantime:
1. Stay hydrated by drinking plenty of water, warm liquids like tea with honey, or coconut water. Avoid caffeine and alcohol as they can dehydrate you further.
2. Gargle with a mixture of warm water and salt to help soothe your throat and reduce inflammation. You can also try gargling with apple cider vinegar diluted in water for potential antibacterial benefits.
3. Use over-the-counter pain relievers li

"Hello! I'm Bioinsight, here to help you with your health concerns. Your sore throat has been persisting for weeks now, which could be due to various reasons. Some possible causes include viral infections like the common cold or flu, bacterial infections such as strep throat, allergies, acid reflux, postnasal drip, and even some more serious conditions like mononucleosis or oral cancer. It's essential to consult a healthcare professional for an accurate diagnosis and appropriate treatment.\n\nHowever, there are several home remedies that might help alleviate your symptoms in the meantime:\n1. Stay hydrated by drinking plenty of water, warm liquids like tea with honey, or coconut water. Avoid caffeine and alcohol as they can dehydrate you further.\n2. Gargle with a mixture of warm water and salt to help soothe your throat and reduce inflammation. You can also try gargling with apple cider vinegar diluted in water for potential antibacterial benefits.\n3. Use over-the-counter pain reliev

In [2]:
import json
import csv

def jsonl_to_csv(jsonl_file, csv_file):
    with open(jsonl_file, 'r', encoding='utf-8') as json_file, open(csv_file, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Question', 'Answer'])  # Write header

        for line in json_file:
            data = json.loads(line)
            csv_writer.writerow([data['Question'], data['Answer']])

# Replace 'input.jsonl' and 'output.csv' with your file names
jsonl_to_csv('merged.jsonl', 'output.csv')


In [7]:
import pandas as pd
df = pd.read_csv("output.csv")
print(df)

                                         id  \
0      3c1f17c3-87e4-48e3-b0ca-19e4f5e9968c   
1      475c4254-0dd9-4c14-88c2-6fda785c72dd   
2      83975b32-644c-4676-bac2-8830acf941b4   
3      e4eba55c-3acf-43cd-afa7-002a1c155535   
4      f69e8f5d-5b09-4081-953e-730abacf98e8   
...                                     ...   
60092  5f277cf7-594b-4a0a-974d-71a31bc3d6f1   
60093  5548cd33-dc6e-45c9-a4fa-e5868cb074bc   
60094  4921edd7-c24e-49cf-b2ae-e9a092c4f569   
60095  f02dc337-6a8b-401c-abc2-618b5bc05f6f   
60096  10b2b223-103f-4c63-9b50-70ba545fb92d   

                                                    text  \
0      What are the symptoms of gastritis? However, t...   
1      What does the treatment for gastritis involve?...   
2      How does an upper endoscopy help diagnose gast...   
3      How is a fecal occult blood test (stool test) ...   
4      What is gastritis? !s_sensitive, chron ID: $('...   
...                                                  ...   
60092  How is s

In [5]:
import json
import csv
from uuid import uuid4  # for generating unique IDs

def jsonl_to_csv(jsonl_file, csv_file):
    with open(jsonl_file, 'r') as json_file, open(csv_file, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['id', 'text', 'metadata'])  # Updated header

        for line in json_file:
            data = json.loads(line)
            
            # Generate a unique ID for each row
            row_id = str(uuid4())

            # Merge 'Question' and 'Answer' under a new 'text' column
            text = f"{data['Question']} {data['Answer']}"

            # Add a metadata column with the specified value
            metadata = json.dumps({"source": "merged.json"})

            # Write the row to the CSV file
            csv_writer.writerow([row_id, text, metadata])


# Replace 'input.jsonl' and 'output.csv' with your file names
jsonl_to_csv('merged.jsonl', 'output.csv')


In [8]:
import csv
import json

def csv_to_json(csv_file, json_file):
    with open(csv_file, 'r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = list(csv_reader)

    with open(json_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=2)

# Replace 'input.csv' and 'output.json' with your file names
csv_to_json('output.csv', 'final-merged.json')
